<ul>
    <li>Nome: Andrea<\li>
    <li>Cognome: Piancone <\li>
    <li>Matricoa: 812250 <\li>
        </ul>

# Progetto Foundations of Computer Science

<hr>

## Librerie

In [1]:
import pandas as pd
import numpy as np
import datetime
import random
import time
pd.set_option('display.max_columns', None)#rimuovo il limite massimo di colonne visualizzabili
pd.set_option('display.float_format', lambda x: '%.5f' % x) #soppressione della  notazione scientifica

In [2]:
start = time.time()

## Datasets

In [3]:
loan_lenders = pd.read_csv(r'loans_lenders.csv')
loans = pd.read_csv(r'loans.csv')
lenders = pd.read_csv(r'lenders.csv')
country_stats = pd.read_csv(r'country_stats.csv')

<center><h1> Esercizi </h1></center>

## 1) Normalize the loan_lenders table

In [4]:
loan_lenders_normalized = (loan_lenders.set_index(['loan_id'])
                   .apply(lambda x: x.str.split(', ').explode())
                   .reset_index())

loan_lenders_normalized

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


## 2) For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time

In [5]:
loans[['disburse_time', 'planned_expiration_time']] = loans[['disburse_time', 
                                                             'planned_expiration_time']].apply(pd.to_datetime).apply(lambda x: x.dt.date)

loans['duration'] = (loans['planned_expiration_time'] - loans['disburse_time']).dt.days

loans['duration']

0         54.00000
1         96.00000
2         37.00000
3         35.00000
4         58.00000
            ...   
1419602   40.00000
1419603   39.00000
1419604   51.00000
1419605   63.00000
1419606   61.00000
Name: duration, Length: 1419607, dtype: float64

## 3) Find the lenders that have funded at least twice

In [6]:
loans_funded = pd.merge(loans['loan_id'].loc[loans['status'] == 'funded'], loan_lenders_normalized, 
                        on = 'loan_id')

loans_funded['lenders'].value_counts().loc[lambda x: x>=2]

gooddogg1           167512
trolltech4460       141268
gmct                123164
nms                  96713
themissionbeltco     74067
                     ...  
samantha25753010         2
trent5851                2
meghna3445               2
alana5711                2
dave48823877             2
Name: lenders, Length: 893981, dtype: int64

## 4) For each country, compute how many loans have involved that country as borrowers

In [7]:
loans['country_name'].value_counts()

Philippines         285336
Kenya               143699
Peru                 86000
Cambodia             79701
El Salvador          64037
                     ...  
Botswana                 1
Papua New Guinea         1
Mauritania               1
Canada                   1
Uruguay                  1
Name: country_name, Length: 96, dtype: int64

## 5) For each country, compute the overall amount of money borrowed

In [8]:
borrowed_by_country = loans.groupby('country_name')['loan_amount'].sum()

borrowed_by_country

country_name
Afghanistan       1967950.00000
Albania           4307350.00000
Armenia          22950475.00000
Azerbaijan       14784625.00000
Belize             150175.00000
                      ...      
Vietnam          24681100.00000
Virgin Islands      10000.00000
Yemen             3444000.00000
Zambia            1978975.00000
Zimbabwe          5851875.00000
Name: loan_amount, Length: 96, dtype: float64

## 6) Like the previous point, but expressed as a percentage of the overall amount lent

In [9]:
#borrowed_by_country = loans.groupby('country_name')['loan_amount'].sum()

In [10]:
(borrowed_by_country / loans['loan_amount'].sum()) * 100

country_name
Afghanistan      0.16657
Albania          0.36459
Armenia          1.94259
Azerbaijan       1.25141
Belize           0.01271
                   ...  
Vietnam          2.08907
Virgin Islands   0.00085
Yemen            0.29151
Zambia           0.16751
Zimbabwe         0.49532
Name: loan_amount, Length: 96, dtype: float64

## 7) Like the three previous points, but split for each year

In [11]:
loans['disburse_year'] = loans['disburse_time'].apply(lambda x: x.year)

### 7.1) For each country and year, compute how many loans have involved that country as borrowers

In [12]:
loans.groupby(['country_name', 'disburse_year']).size()

country_name  disburse_year
Afghanistan   2007.00000        408
              2008.00000        370
              2009.00000        678
              2010.00000        632
              2011.00000        247
                               ... 
Zimbabwe      2013.00000        426
              2014.00000       2078
              2015.00000        600
              2016.00000        808
              2017.00000       1079
Length: 748, dtype: int64

### 7.2) For each country and year, compute the overall amount of money borrowed

In [13]:
loans.groupby(['country_name', 'disburse_year'])['loan_amount'].sum()

country_name  disburse_year
Afghanistan   2007.00000       194975.00000
              2008.00000       365375.00000
              2009.00000       585125.00000
              2010.00000       563350.00000
              2011.00000       245125.00000
                                   ...     
Zimbabwe      2013.00000       678525.00000
              2014.00000      1311575.00000
              2015.00000       723625.00000
              2016.00000       788600.00000
              2017.00000      1237600.00000
Name: loan_amount, Length: 748, dtype: float64

### 7.3) For each country and year, compute the overall amount of money borrowed

In [14]:
(loans.groupby(['country_name', 'disburse_year'])['loan_amount'].sum() / loans['loan_amount'].sum()) * 100

country_name  disburse_year
Afghanistan   2007.00000      0.01650
              2008.00000      0.03093
              2009.00000      0.04953
              2010.00000      0.04768
              2011.00000      0.02075
                                ...  
Zimbabwe      2013.00000      0.05743
              2014.00000      0.11102
              2015.00000      0.06125
              2016.00000      0.06675
              2017.00000      0.10475
Name: loan_amount, Length: 748, dtype: float64

## 8) For each lender, compute the overall amount of money lent

In [15]:
loans['average_amount'] = loans['loan_amount'] / loans['num_lenders_total']

lent_per_lenders = pd.merge(loan_lenders_normalized, loans[['loan_id', 'average_amount']], 
                              on = 'loan_id')

lent_per_lenders = lent_per_lenders.groupby('lenders')['average_amount'].sum().reset_index(name = 'lent')

lent_per_lenders

,lenders,lent
0,000,1563.88108
1,00000,1249.94736
2,0002,2201.18046
3,00mike00,38.46154
4,0101craign0101,2424.08893
...,...,...
1383794,zzmcfate,60150.83276
1383795,zzpaghetti9994,44.24779
1383796,zzrvmf8538,551.82958
1383797,zzzsai,236.88805


## 9) For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed

In [16]:
def found_distribution(country_code):
    random.seed(1234)
    results = list()
    values = pd.unique(country_code.dropna())
    results.append(values)
    probs = country_code.dropna().value_counts() / len(country_code.dropna())
    results.append(probs)
    index_rows_nan = country_code.index[country_code.isnull()]
    results.append(index_rows_nan)
    n = len(index_rows_nan)
    results.append(n)
    return (results)

In [17]:
results_function = found_distribution(lenders['country_code'])

#results_function[0] = values, results_function[1] = probs, results_function[2] = n, results_function[3] = index_rows_nan

random_country_code = pd.Series(np.random.choice(results_function[0], p = results_function[1],
                                                size = results_function[3]), index = results_function[2])

lenders['country_code'].fillna(random_country_code, inplace = True)

In [18]:
borrowed_by_country = loans.groupby('country_code')['loan_amount'].sum().reset_index(name = 'borrowed')

lent_per_lenders = pd.merge(lent_per_lenders, lenders[['permanent_name', 'country_code']], 
                             left_on = 'lenders', right_on = 'permanent_name')

lent_per_country = lent_per_lenders.groupby('country_code')['lent'].sum().reset_index(name = 'lent')

lent_borrowed_by_country = pd.merge(lent_per_country, borrowed_by_country, on = 'country_code', how = 'outer')

lent_borrowed_by_country.fillna(0, inplace = True)

lent_borrowed_by_country['difference'] = lent_borrowed_by_country['lent'] - lent_borrowed_by_country['borrowed']

lent_borrowed_by_country

,country_code,lent,borrowed,difference
0,AD,23567.03893,0.00000,23567.03893
1,AE,1693764.03131,0.00000,1693764.03131
2,AF,109822.89614,1967950.00000,-1858127.10386
3,AG,596.99757,0.00000,596.99757
4,AI,916.53300,0.00000,916.53300
...,...,...,...,...
229,YT,6671.22923,0.00000,6671.22923
230,ZA,507443.76066,1006525.00000,-499081.23934
231,ZM,37965.51223,1978975.00000,-1941009.48777
232,ZW,7999.51499,5851875.00000,-5843875.48501


## 10) Which country has the highest ratio between the difference computed at the previous point and the population

In [19]:
lent_borrowed_by_country = pd.merge(lent_borrowed_by_country, country_stats[['country_code', 'population', 
                                                                             'population_below_poverty_line']], 
                                    on = 'country_code')

lent_borrowed_by_country = lent_borrowed_by_country.set_index('country_code')

(lent_borrowed_by_country['difference'] / lent_borrowed_by_country['population']).idxmax()

'NO'

## 11) Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line

In [20]:
lent_borrowed_by_country['population_not_below_poverty_line'] = 100 - lent_borrowed_by_country['population_below_poverty_line']

lent_borrowed_by_country['abs_population_not_below_poverty_line'] = (lent_borrowed_by_country['population'] *
                                                                    lent_borrowed_by_country['population_not_below_poverty_line']) / 100
  
(lent_borrowed_by_country['difference'] / lent_borrowed_by_country['abs_population_not_below_poverty_line']).idxmax()

'US'

## 12) For each year, compute the total amount of loans

In [21]:
loans['duration'] = loans['duration'].abs()

loans_not_null = loans.dropna(subset = ['planned_expiration_time', 'disburse_time'])

# Per questioni computazionali separo i prestiti con duration uguale a zero con i prestati con duration maggiore di zero

loans__duration_not_zero = loans_not_null[loans_not_null['duration'] != 0]

loans_duration_zero = loans_not_null[loans_not_null['duration'] == 0]

In [22]:
def day_count(date1, date2):
    if date1 < date2:
        begin = date1 + datetime.timedelta(1) #poichè sono presenti presenti con disbirse time antecendente a planned_expiration_time
        end = date2 + datetime.timedelta(1)   # è stato necessario inserire una struttura di controllo.
    else:
        begin = date2 + datetime.timedelta(1)
        end = date1 + datetime.timedelta(1)
    years = range(begin.year,end.year + 1)
    dictionary = dict()
    for year in years:
        year_start = datetime.date(year, 1, 1)
        year_end = datetime.date(year + 1, 1, 1)
        dictionary.update({year:(min(end, year_end) - max(begin, year_start)).days})
    return pd.Series(dictionary, name = 'Date_Value')

In [23]:
timetable = loans__duration_not_zero.apply(lambda x: day_count(x['disburse_time'], x['planned_expiration_time']), 
                                 axis = 1)

timetable

,2011,2012,2013,2014,2015,2016,2017,2018
0,nan,nan,9.00000,45.00000,nan,nan,nan,nan
1,nan,nan,11.00000,85.00000,nan,nan,nan,nan
2,nan,nan,nan,37.00000,nan,nan,nan,nan
3,nan,nan,nan,35.00000,nan,nan,nan,nan
4,nan,nan,14.00000,44.00000,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...
1419602,nan,nan,nan,nan,38.00000,2.00000,nan,nan
1419603,nan,nan,nan,nan,37.00000,2.00000,nan,nan
1419604,nan,nan,nan,nan,48.00000,3.00000,nan,nan
1419605,nan,nan,nan,nan,58.00000,5.00000,nan,nan


In [24]:
timetable.fillna(0, inplace = True)

timetable = timetable.set_index(loans__duration_not_zero['loan_id']).reset_index()

timetable = pd.merge(timetable, loans__duration_not_zero[['loan_id', 'loan_amount', 'duration']], on = 'loan_id')

timetable

,loan_id,2011,2012,2013,2014,2015,2016,2017,2018,loan_amount,duration
0,657307,0.00000,0.00000,9.00000,45.00000,0.00000,0.00000,0.00000,0.00000,125.00000,54.00000
1,657259,0.00000,0.00000,11.00000,85.00000,0.00000,0.00000,0.00000,0.00000,400.00000,96.00000
2,658010,0.00000,0.00000,0.00000,37.00000,0.00000,0.00000,0.00000,0.00000,400.00000,37.00000
3,659347,0.00000,0.00000,0.00000,35.00000,0.00000,0.00000,0.00000,0.00000,625.00000,35.00000
4,656933,0.00000,0.00000,14.00000,44.00000,0.00000,0.00000,0.00000,0.00000,425.00000,58.00000
...,...,...,...,...,...,...,...,...,...,...,...
1044213,988180,0.00000,0.00000,0.00000,0.00000,38.00000,2.00000,0.00000,0.00000,400.00000,40.00000
1044214,988213,0.00000,0.00000,0.00000,0.00000,37.00000,2.00000,0.00000,0.00000,300.00000,39.00000
1044215,989109,0.00000,0.00000,0.00000,0.00000,48.00000,3.00000,0.00000,0.00000,2425.00000,51.00000
1044216,989143,0.00000,0.00000,0.00000,0.00000,58.00000,5.00000,0.00000,0.00000,100.00000,63.00000


In [25]:
loan_amount_per_year = timetable[[2011,2012,2013,2014,2015,2016,2017,2018
                        ]].div(timetable['duration'], axis=0).mul(timetable['loan_amount'], axis = 0)

In [26]:
df1 = loan_amount_per_year.sum().to_frame().reset_index().rename(columns = {'index':'year', 0:'loan_amount'})

In [27]:
df2 = loans_duration_zero.groupby('disburse_year')['loan_amount'].sum().to_frame().reset_index().rename(columns = {'disburse_year':'year'})

In [28]:
loan_amount_per_year_complete = pd.concat([df1,df2]).groupby('year')['loan_amount'].sum().to_frame().reset_index()

loan_amount_per_year_complete

,year,loan_amount
0,2011.00000,629911.77488
1,2012.00000,113287580.79478
2,2013.00000,125015552.04954
3,2014.00000,153214272.06143
4,2015.00000,159929916.50857
5,2016.00000,159363790.44077
6,2017.00000,172536423.51457
7,2018.00000,5826652.85546


In [29]:
#si verifica se la distrubuzione dell'ammontare dei prestiti è andata a buon fine

loan_amount_per_year_complete['loan_amount'].sum() - loans_not_null['loan_amount'].sum()

-1.1920928955078125e-06

In [30]:
# Poichè il delta tra i due ammontari è praticamente nullo si può concludere che la distribuzione dell'ammontare dei prestiti è 
# andata a buon fine

In [31]:
end = time.time()

print("This script took: {} minutes".format(int((end-start)/60)))

This script took: 11 minutes
